### Code for the coursework for the Virtual and Augmented Reality submodule in the Contemporary Computer Science IV module
Libraries used: python (3.7.2), pandas, matplotlib


###### To run this code, please follow these instructions:
The code works best when executed in order (top to bottom), one cell at a time. The matplotlib outputs are 
given in pop-out windows, but the 3D (animated) plots seem to replace one-another and the previous outputs
when they are produced. Thus, to see all output correctly, the cells should be run one at a time.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D

# Create aliases for the headings of the IMU data for easier processing
time   = 'time'
gyro_X = ' gyroscope.X'
gyro_Y = ' gyroscope.Y'
gyro_Z = ' gyroscope.Z'
acc_X  = ' accelerometer.X'
acc_Y  = ' accelerometer.Y'
acc_Z  = ' accelerometer.Z'
mag_X  = ' magnetometer.X'
mag_Y  = ' magnetometer.Y'
mag_Z  = ' magnetometer.Z '

# NOTE:
# X is 'depth' (forward/backward) (i.e. the axis of roll)
# Y is 'across' (left/right) (i.e. the axis of pitch)
# Z is 'up' (up/down) (i.e. the axis of yaw)


In [ ]:
# Problem 1

# Read the IMU data from the .csv file
IMU_Data = pd.read_csv("IMUData.csv")
# Create raw acc reading values to be used for Problem 6
IMU_Data['raw_accX'] = IMU_Data[acc_X]
IMU_Data['raw_accY'] = IMU_Data[acc_Y]
IMU_Data['raw_accZ'] = IMU_Data[acc_Z]

# Convert rotational rate from deg/s to rad/s
IMU_Data[gyro_X] = IMU_Data[gyro_X].apply(np.radians)
IMU_Data[gyro_Y] = IMU_Data[gyro_Y].apply(np.radians)
IMU_Data[gyro_Z] = IMU_Data[gyro_Z].apply(np.radians)

# Normalize the accelerometer and magnetometer values
def normalize(x, y, z):
    """Normalizes the magnitude of the given [X, Y, Z] vector
    
    \in x: the x-axis value
    \in y: the y-axis value
    \in z: the z-axis value
    \out x_norm: the normalised x-axis value
    \out y_norm: the normalised y-axis value
    \out z_norm: the normalised z-axis value
    """
    magnitude = np.sqrt(x**2 + y**2 + z**2)
    
    # Only normalize if magnitude > 0
    if magnitude:
        return x/magnitude, y/magnitude, z/magnitude
    
    # If magnitude == 0, return the vector as-is
    return x, y, z

for index, row in IMU_Data.iterrows():
    row[acc_X], row[acc_Y], row[acc_Z] = normalize(row[acc_X], row[acc_Y], row[acc_Z])
    row[mag_X], row[mag_Y], row[mag_Z] = normalize(row[mag_X], row[mag_Y], row[mag_Z])

# Methods for converting Euler angles (radians) <---> quaternions

def euler2quaternion(eulers):
    """Converts the given euler angles into a quaternion format
    
    \in eulers: the input euler angles, of format (x, y, z), equivalently (roll, pitch, yaw)
    \out quat: the equivalent quaternion, of format (w, x, y, z)
    """
    # extract the euler angles from eulers
    roll, pitch, yaw = eulers
    
    # Compute the sinusoidal values needed
    cos_y = np.cos(yaw/2)
    sin_y = np.sin(yaw/2)
    cos_p = np.cos(pitch/2)
    sin_p = np.sin(pitch/2)
    cos_r = np.cos(roll/2)
    sin_r = np.sin(roll/2)
    
    # Compute the quaternion values
    quat_w = cos_y * cos_p * cos_r + sin_y * sin_p * sin_r
    quat_x = cos_y * cos_p * sin_r - sin_y * sin_p * cos_r
    quat_y = sin_y * cos_p * sin_r + cos_y * sin_p * cos_r
    quat_z = sin_y * cos_p * cos_r - cos_y * sin_p * sin_r
    
    return (quat_w, quat_x, quat_y, quat_z)

def quaternion2euler(quat):
    """Converts the given quaternion into euler angle format
    
    \in quat: the input quaternion, of format (w, x, y, z)
    \out euler: the equivalent euler angles, of format (x, y, z), equivalently (roll, pitch, yaw)
    """
    # extract the quaternion values from quat
    w, x, y, z = quat
    
    # Compute the roll
    roll  = np.arctan2(2 * (w*x + y*z), 1 - 2 * (x**2 + y**2))
    
    # Compute the pitch
    asin_val = 2 * (w*y - z*x)
    # Check if this value is out of the range of arcsin (-1, 1)
    if abs(asin_val) >= 1:
        # Use pi/2 as the value as the value is out of the range of arcsin
        sign = 1 if asin_val >= 0 else -1
        pitch = np.pi/2 * sign
    else:
        pitch = np.arcsin(asin_val)
    
    # Compute the yaw
    yaw = np.arctan2(2 * (w*z + x*y), 1 - 2 * (y**2 + z**2))
    
    return (roll, pitch, yaw)
    

# Method for converting quaternion to its conjugate
def quaternion_conjugate(quat):
    """Convert the given quaternion to its conjugate
    
    \in quat: the input quaternion, of format (w, x, y, z)
    \out conj_quat: the conjugate of quat, of format (w, x, y, z)
    """
    return (quat[0], -quat[1], -quat[2], -quat[3])

# Method for calculating quaternion product of two quaternions
def quaternion_product(quat1, quat2):
    """Calculate the product of the two quaternions
    
    \in quat1: the first quaternion, of format (w, x, y, z)
    \in quat2: the second quaternion, of format (w, x, y, z)
    \out quat: the product of quat1 and quat2, of format (w, x, y, z)
    """
    # Extract the quaternion values
    w1, x1, y1, z1 = quat1
    w2, x2, y2, z2 = quat2
    
    # Construct the quaternion product
    w = w1*w2 - x1*x2 - y1*y2 - z1*z2
    x = w1*x2 + x1*w2 + y1*z2 - z1*y2
    y = w1*y2 - x1*z2 + y1*w2 + z1*x2
    z = w1*z2 + x1*y2 - y1*x2 + z1*w2
    
    return (w, x, y, z)

# General (non-required) utility functions
def quaternion_rotation(q, p):
    """Rotates the point p by the quaternion q
    
    \in q: the quaternion of the rotation
    \in p: the point to be rotated by q
    \out result: the point produced by the rotation"""
    if len(p) == 3:
        p = (0, p[0], p[1], p[2])
    result = quaternion_product(p, quaternion_conjugate(q))
    result = quaternion_product(q, result)
    return result

In [ ]:
# Problem 2

# Create the DataFrame used to store the head orientations
head_orientation = pd.DataFrame(columns=['time', 'quaternion', 'W', 'X', 'Y', 'Z']) # List of quaternions corresponding to orientation at each time stamp
head_orientation['time'] = IMU_Data['time'][:]

def simple_dead_reckoning(prev_ho_row, current_IMU_Data_row):
    """Performs simple dead reckoning to track the current head orientation by integrating
    the gyroscope readings.
    
    \in prev_ho_row: the previous head_orientation row
    \in current_IMU_Data_row: the row of data in the IMU_Data frame corresponding to the current time step
    \out current_orientation: the estimated current head orientation
    """
    # Extract the data we need from the current_IMU_Data_row (row in IMU_Data frame that corresponds to the current time step)
    current_time = current_IMU_Data_row[time]
    gyroX        = current_IMU_Data_row[gyro_X]
    gyroY        = current_IMU_Data_row[gyro_Y]
    gyroZ        = current_IMU_Data_row[gyro_Z]
    
    # Extract the data we need from the prev_ho_row (previous row in the head_orientation_tilt_correction frame)
    last_orientation = prev_ho_row['quaternion']
    last_time = prev_ho_row[time]
    
    # Calculate the magnitude of the gyro readings
    l = np.sqrt(gyroX ** 2 + gyroY ** 2 + gyroZ ** 2)
    
    # Normalize the gyro readings (so they can be used as axes in axis-angle representation)
    gyroX /= l
    gyroY /= l
    gyroZ /= l
    
    # Calculate the angle of the rotation
    theta = l*(current_time - last_time)
    
    # Create the quaternion from this axis-angle representation
    change_quat = (np.cos(theta/2), gyroX*np.sin(theta/2), gyroY*np.sin(theta/2), gyroZ*np.sin(theta/2))

    # Produce the current orientation by combining (multiplying) the last orientation and the change quaternion
    current_orientation = quaternion_product(last_orientation, change_quat)
    
    return current_orientation

# Initial orientation
head_orientation.at[0, 'quaternion'] = (1, 0, 0, 0)
for index in range(len(head_orientation)):
    # Skip the first one as this has the value of the identity quaternion
    if index != 0:
        prev_row = head_orientation.iloc[index - 1]
        imu_row = IMU_Data.iloc[index]
        # Estimate the current head orientation
        head_orientation.at[index, 'quaternion'] = simple_dead_reckoning(prev_row, imu_row)
    # Set the separate W, X, Y, Z values of this row
    quat = head_orientation.iloc[index]['quaternion']
    head_orientation.at[index, 'W'] = quat[0]
    head_orientation.at[index, 'X'] = quat[1]
    head_orientation.at[index, 'Y'] = quat[2]
    head_orientation.at[index, 'Z'] = quat[3]

# Convert the quaternion orientations to Euler angles for 2D plotting in Problem 5
head_orientation_euler = pd.DataFrame(columns=['time', 'X', 'Y', 'Z'])
head_orientation_euler['time'] = IMU_Data['time'][:]
for index in range(len(head_orientation)):
    quat = head_orientation.iloc[index]['quaternion']
    x, y, z = quaternion2euler(quat)
    head_orientation_euler.at[index, 'X'] = x
    head_orientation_euler.at[index, 'Y'] = y
    head_orientation_euler.at[index, 'Z'] = z

In [ ]:
# Problem 3

# Create the DataFrame used to store the head orientations
head_orientation_tilt_correction = pd.DataFrame(columns=['time', 'quaternion', 'W', 'X', 'Y', 'Z']) # List of quaternions corresponding to orientation at each time stamp
head_orientation_tilt_correction['time'] = IMU_Data['time'][:]

def tilt_correction(prev_ho_row, current_IMU_Data_row):
    """Estimates the head orientation with additionally applying gravity-based tilt correction
    
    \in prev_ho_row: the row of data (time and quaternion) corresponding to the last time step
    \in current_IMU_Data_row: the row of data in the IMU_Data frame that corresponds to the current time step
    \out tilt_corrected_orientation: the estimated head orientation, with gravity-based tilt correction
    """
    # Extract the data we need from current_IMU_Data_row
    accX = current_IMU_Data_row[acc_X]
    accY = current_IMU_Data_row[acc_Y]
    accZ = current_IMU_Data_row[acc_Z]
        
    # Get the result from simple dead reckoning
    simple_orientation = simple_dead_reckoning(prev_ho_row, current_IMU_Data_row)
    
    # Perform tilt correction
    
    # Transform acceleration measurements into the global frame
    acc_quat = (0, accX, accY, accZ) # convert the acc vector into a quaternion as per LaValle paper
    a_global = quaternion_rotation(simple_orientation, acc_quat)
    
    # Calculate the tilt axis - this is orthogonal to the XY plane (as Z is 'up'), so we take it as (-a_y, a_x, 0)
    tilt_axis = (-a_global[2], a_global[1], 0)
    
    # Find the angle phi between the up vector (0, 0, 1) and the accelerometer vector (a_x, a_y, a_z)
    # phi = arccos(dot_product(a, up)/(magnitude(a)*magnitude(up))). However, magnitude of both a and up are 1, as a is normalized
    # and up is (0, 0, 1). Also, since up is (0, 0, 1) the dot product of a and up is simply a_z. Thus:
    phi = np.arccos(a_global[3])
    
    # Use the complementary filter to fuse the two signals
    
    # Construct the tilt quaternion
    alpha = 0.03
    theta = -alpha * phi

    tilt_quat = (np.cos(theta/2), tilt_axis[0]*np.sin(theta/2), tilt_axis[1]*np.sin(theta/2), tilt_axis[2]*np.sin(theta/2))
    
    tilt_corrected_orientation = quaternion_product(tilt_quat, simple_orientation)
    
    return tilt_corrected_orientation

# Initial orientation
head_orientation_tilt_correction.at[0, 'quaternion'] = (1, 0, 0, 0)
for index in range(len(head_orientation_tilt_correction)):
    # Skip the first one as this has the value of the identity quaternion
    if index != 0:
        prev_row = head_orientation_tilt_correction.iloc[index - 1]
        imu_row = IMU_Data.iloc[index]
        # Estimate the current head orientation, with tilt correction
        head_orientation_tilt_correction.at[index, 'quaternion'] = tilt_correction(prev_row, imu_row)
    # Set the separate W, X, Y, Z values of this row
    quat = head_orientation_tilt_correction.iloc[index]['quaternion']
    head_orientation_tilt_correction.at[index, 'W'] = quat[0]
    head_orientation_tilt_correction.at[index, 'X'] = quat[1]
    head_orientation_tilt_correction.at[index, 'Y'] = quat[2]
    head_orientation_tilt_correction.at[index, 'Z'] = quat[3]

# Convert the quaternion orientations to Euler angles for 2D plotting in Problem 5
head_orientation_tilt_correction_euler = pd.DataFrame(columns=['time', 'X', 'Y', 'Z'])
head_orientation_tilt_correction_euler['time'] = IMU_Data['time'][:]
for index in range(len(head_orientation_tilt_correction)):
    quat = head_orientation_tilt_correction.iloc[index]['quaternion']
    x, y, z = quaternion2euler(quat)
    head_orientation_tilt_correction_euler.at[index, 'X'] = x
    head_orientation_tilt_correction_euler.at[index, 'Y'] = y
    head_orientation_tilt_correction_euler.at[index, 'Z'] = z

In [ ]:
# Problem 4

# Create the DataFrame used to store the head orientations
head_orientation_yaw_correction = pd.DataFrame(columns=['time', 'quaternion', 'W', 'X', 'Y', 'Z']) # List of quaternions corresponding to orientation at each time stamp
head_orientation_yaw_correction['time'] = IMU_Data['time'][:]

def yaw_correction(prev_ho_row, current_IMU_Data_row):
    """Estimates the head orientation with additionally applying gravity-based tilt correction
    
    \in prev_ho_row: the row of data (time and quaternion) corresponding to the last time step
    \in current_IMU_Data_row: the row of data in the IMU_Data frame that corresponds to the current time step
    \out tilt_corrected_orientation: the estimated head orientation, with gravity-based tilt correction
    """
    # Extract the data we need from current_IMU_Data_row
    magX = current_IMU_Data_row[mag_X]
    magY = current_IMU_Data_row[mag_Y]
    magZ = current_IMU_Data_row[mag_Z]
    
    # Get the result from tilt correction
    tilt_corrected_orientation = tilt_correction(prev_ho_row, current_IMU_Data_row)
    
    # Perform yaw correction
    
    # Get reference readings
    mag_ref = (0, IMU_Data.iloc[0][mag_X], IMU_Data.iloc[0][mag_Y], IMU_Data.iloc[0][mag_Z])
    quat_ref = head_orientation_yaw_correction.iloc[0]['quaternion']
    
    mag_current = (0, magX, magY, magZ)
    
    # Bring the current readings into the estimated global frame
    q = tilt_corrected_orientation
    q_inv = quaternion_conjugate(tilt_corrected_orientation)
    mag_current = quaternion_product(q, quaternion_product(mag_current, q_inv))
    
    # Find the angular difference between mag_current and mag_ref, when projected into the XY plane
    theta_current = np.arctan2(mag_current[1], mag_current[2])
    theta_ref = np.arctan2(mag_ref[1], mag_ref[2])
    
    # Apply the complementary filter to fix the yaw drift
    alpha = 0.0002
    # Construct the correction quaternion
    theta = -alpha * (theta_current - theta_ref)
    correction_quat = (np.cos(theta/2), 0, 0, np.sin(theta/2))
    
    yaw_corrected_orientation = quaternion_product(correction_quat, tilt_corrected_orientation) #if (theta_current - theta_ref) > 
    
    return yaw_corrected_orientation

# Initial orientation
head_orientation_yaw_correction.at[0, 'quaternion'] = (1, 0, 0, 0)
for index in range(len(head_orientation_yaw_correction)):
    # Skip the first one as this has the value of the identity quaternion
    if index != 0:
        prev_row = head_orientation_yaw_correction.iloc[index - 1]
        imu_row = IMU_Data.iloc[index]
        # Estimate the current head orientation, with tilt correction
        head_orientation_yaw_correction.at[index, 'quaternion'] = yaw_correction(prev_row, imu_row)
    # Set the separate W, X, Y, Z values of this row
    quat = head_orientation_yaw_correction.iloc[index]['quaternion']
    head_orientation_yaw_correction.at[index, 'W'] = quat[0]
    head_orientation_yaw_correction.at[index, 'X'] = quat[1]
    head_orientation_yaw_correction.at[index, 'Y'] = quat[2]
    head_orientation_yaw_correction.at[index, 'Z'] = quat[3]

# Convert the quaternion orientations to Euler angles for 2D plotting in Problem 5
head_orientation_yaw_correction_euler = pd.DataFrame(columns=['time', 'X', 'Y', 'Z'])
head_orientation_yaw_correction_euler['time'] = IMU_Data['time'][:]

for index in range(len(head_orientation_yaw_correction)):
    quat = head_orientation_yaw_correction.iloc[index]['quaternion']
    x, y, z = quaternion2euler(quat)
    head_orientation_yaw_correction_euler.at[index, 'X'] = x
    head_orientation_yaw_correction_euler.at[index, 'Y'] = y
    head_orientation_yaw_correction_euler.at[index, 'Z'] = z


In [ ]:
# Problem 5 - Gyroscope Readings

# Make plots be produced in pop-out windows and be interactive
%matplotlib qt
# Tri-axial angular rate in deg/s
fig = figure(figsize=(24, 8))
fig.canvas.set_window_title('Gyroscope Readings')

plt.plot(IMU_Data[time], np.degrees(IMU_Data[gyro_X]), label='X')
plt.plot(IMU_Data[time], np.degrees(IMU_Data[gyro_Y]), label='Y')
plt.plot(IMU_Data[time], np.degrees(IMU_Data[gyro_Z]), label='Z')

plt.title("Tri-axial angular rate (Gyroscope)")
plt.xlabel("Time (s)")
plt.ylabel("Angular rate ($deg/s$)")
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Problem 5 - Accelerometer Readings

# Make plots be produced in pop-out windows and be interactive
%matplotlib qt
# Normalized tri-axial acceleration in g (m/s^2)
fig = figure(figsize=(24, 8))
fig.canvas.set_window_title('Normalized Accelerometer Readings')
plt.plot(IMU_Data[time], IMU_Data[acc_X], label='X')
plt.plot(IMU_Data[time], IMU_Data[acc_Y], label='Y')
plt.plot(IMU_Data[time], IMU_Data[acc_Z], label='Z')

plt.title("Normalized tri-axial acceleration (Accelerometer)")
plt.xlabel("Time (s)")
plt.ylabel("Acceleration in $g$ ($m/s^2$)")
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Problem 5 - Magnetometer Readings

# Make plots be produced in pop-out windows and be interactive
%matplotlib qt
# Normalized tri-axial magnetic flux in Gauss (G)
fig = figure(figsize=(24, 8))
fig.canvas.set_window_title('Normalized Magnetometer Readings')
plt.plot(IMU_Data[time], IMU_Data[mag_X], label='X')
plt.plot(IMU_Data[time], IMU_Data[mag_Y], label='Y')
plt.plot(IMU_Data[time], IMU_Data[mag_Z], label='Z')

plt.title("Normalized tri-axial magnetic flux (Magnetometer)")
plt.xlabel("Time (s)")
plt.ylabel("Magnetic flux in Gauss ($G$)")
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Problem 5 - Euler Angles of Head Orientation

# Tri-axial Euler angles in degrees for gyroscope integration, gyroscope + accelerometer, gyroscope + accelerometer + magnetometer

fig = figure(figsize=(24, 12))
fig.canvas.set_window_title('Tri-axial Euler angles (deg) for 3 types of orientation tracking')
plt.tight_layout()
plt.subplot(3, 1, 1)
# Gyroscope
plt.plot(head_orientation_euler[time], np.degrees(np.float32(head_orientation_euler['X'])), label='X')
plt.plot(head_orientation_euler[time], np.degrees(np.float32(head_orientation_euler['Y'])), label='Y')
plt.plot(head_orientation_euler[time], np.degrees(np.float32(head_orientation_euler['Z'])), label='Z')
plt.axhline(0, color='black', alpha=0.3)
plt.title("Gyroscope Integration")
plt.xlabel("Time (s)")
plt.ylabel("Euler angle (deg)")
plt.legend(loc='upper right')

plt.subplot(3, 1, 2)
# Gyroscope + Accelerometer
plt.plot(head_orientation_tilt_correction_euler[time], np.degrees(np.float32(head_orientation_tilt_correction_euler['X'])), label='X')
plt.plot(head_orientation_tilt_correction_euler[time], np.degrees(np.float32(head_orientation_tilt_correction_euler['Y'])), label='Y')
plt.plot(head_orientation_tilt_correction_euler[time], np.degrees(np.float32(head_orientation_tilt_correction_euler['Z'])), label='Z')
plt.axhline(0, color='black', alpha=0.3)
plt.title("Gyroscope + Accelerometer")
plt.xlabel("Time (s)")
plt.ylabel("Euler angle (deg)")
plt.legend(loc='upper right')

plt.subplot(3, 1, 3)
# Gyroscope + Accelerometer + Magnetometer
plt.plot(head_orientation_yaw_correction_euler[time], np.degrees(np.float32(head_orientation_yaw_correction_euler['X'])), label='X')
plt.plot(head_orientation_yaw_correction_euler[time], np.degrees(np.float32(head_orientation_yaw_correction_euler['Y'])), label='Y')
plt.plot(head_orientation_yaw_correction_euler[time], np.degrees(np.float32(head_orientation_yaw_correction_euler['Z'])), label='Z')
plt.axhline(0, color='black', alpha=0.3)
plt.title("Gyroscope + Accelerometer + Magnetometer")
plt.xlabel("Time (s)")
plt.ylabel("Euler angle (deg)")
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
# Problem 5 - 3D plots full speed
# 3D plots of the 3 main axis of the IMU for gyroscope integration, gyroscope + acceleration, gyroscope + acceleration + magnetometer

# Make plots pop-out and be interactive
%matplotlib qt

N = 6959 # Number of frames
interval = 4 # Time between frames in ms - 1000/256 as time steps are at 256Hz
resolution = 25 # How often frames are generated (i.e. resolution=4 means every 4th data point is plotted)
# The resolution variable is used to ensure the animation is full speed/half speed as required. With the given values it runs at the correct speeds on my computer, but if yours is faster/slower the actual speeds will likely differ

N //= resolution # Set the total number of frames as adjusted by resolution
interval *= resolution # Set the time between frames as adjusted by resolution
scale = 2

axisX, axisY, axisZ = [0, 0, 0], [0, 0, 0], [0, 0, 0]

def animate_axes(num, ax1, ax2, ax3):
    """Generate the new quivers"""
    global axisX, axisY, axisZ
    # Check that this isn't the first frame
    if axisX[0] != 0:
        for i in axisX, axisY, axisZ:
            for j in i:
                j.remove()
    
    num *= resolution
    
    x_proj_vec = (0, 1, 0, 0)
    y_proj_vec = (0, 0, 1, 0)
    z_proj_vec = (0, 0, 0, 1)
    # Apply the current quaternion (indexed by num) to these vectors
    quaternion = head_orientation.iloc[num]['quaternion']
    tilt_corrected_quaternion = head_orientation_tilt_correction.iloc[num]['quaternion']
    yaw_corrected_quaternion = head_orientation_yaw_correction.iloc[num]['quaternion']
    
    axes = [ax1, ax2, ax3]
    quats = [quaternion, tilt_corrected_quaternion, yaw_corrected_quaternion]
    
    for i in range(3):
        ax = axes[i]
        quat = quats[i]

        x_vec = quaternion_rotation(quat, x_proj_vec)
        y_vec = quaternion_rotation(quat, y_proj_vec)
        z_vec = quaternion_rotation(quat, z_proj_vec)

        axisX[i] = ax.quiver(0, 0, 0, *x_vec[1:], color='Blue', label='X')
        axisY[i] = ax.quiver(0, 0, 0, *y_vec[1:], color='Orange', label='Y')
        axisZ[i] = ax.quiver(0, 0, 0, *z_vec[1:], color='Green', label='Z')
        
        ax.legend(loc='center left')

fig = plt.figure(figsize=(24, 8))
fig.canvas.set_window_title('Orientation Tracking - Full speed')
# Gyroscope integration
ax1 = fig.add_subplot(131, projection='3d')
ax1.set_title("Gyroscope integration")
ax1.set_xlim3d([-1.0*scale, 1.0*scale]); ax1.set_xlabel('X')
ax1.set_ylim3d([-1.0*scale, 1.0*scale]); ax1.set_ylabel('Y')
ax1.set_zlim3d([-1.0*scale, 1.0*scale]); ax1.set_zlabel('Z')

# Gyroscope + Accelerometer
ax2 = fig.add_subplot(132, projection='3d')
ax2.set_title("Gyroscope + Accelerometer")
ax2.set_xlim3d([-1.0*scale, 1.0*scale]); ax2.set_xlabel('X')
ax2.set_ylim3d([-1.0*scale, 1.0*scale]); ax2.set_ylabel('Y')
ax2.set_zlim3d([-1.0*scale, 1.0*scale]); ax2.set_zlabel('Z')

# Gyroscope + Accelerometer + Magnetometer
ax3 = fig.add_subplot(133, projection='3d')
ax3.set_title("Gyroscope + Accelerometer + Magnetometer")
ax3.set_xlim3d([-1.0*scale, 1.0*scale]); ax3.set_xlabel('X')
ax3.set_ylim3d([-1.0*scale, 1.0*scale]); ax3.set_ylabel('Y')
ax3.set_zlim3d([-1.0*scale, 1.0*scale]); ax3.set_zlabel('Z')

ani = animation.FuncAnimation(fig, animate_axes, N, fargs=(ax1, ax2, ax3), interval=interval, blit=False)

plt.tight_layout()
plt.show()

In [ ]:
# Problem 5 - 3D plots half speed
# 3D plots of the 3 main axis of the IMU for gyroscope integration, gyroscope + acceleration, gyroscope + acceleration + magnetometer

# Make plots pop-out and be interactive
%matplotlib qt

# Redefine all variables used above so this cell can be called several times in a row
N = 6959 # Number of frames
interval = 4 # Time between frames in ms - 1000/256 as time steps are at 256Hz
resolution = 12 # How often frames are generated (i.e. resolution=4 means every 4th data point is plotted)
# The resolution variable is used to ensure the animation is full speed/half speed as required. With the given values it runs at the correct speeds on my computer, but if yours is faster/slower the actual speeds will likely differ

N //= resolution # Set the total number of frames as adjusted by resolution
interval *= resolution # Set the time between frames as adjusted by resolution
scale = 2

axisX, axisY, axisZ = [0, 0, 0], [0, 0, 0], [0, 0, 0]

def animate_axes(num, ax1, ax2, ax3):
    """Generate the new quivers"""
    global axisX, axisY, axisZ
    # Check that this isn't the first frame
    if axisX[0] != 0:
        for i in axisX, axisY, axisZ:
            for j in i:
                j.remove()
        
    num *= resolution
    
    x_proj_vec = (0, 1, 0, 0)
    y_proj_vec = (0, 0, 1, 0)
    z_proj_vec = (0, 0, 0, 1)
    # Apply the current quaternion (indexed by num) to these vectors
    quaternion = head_orientation.iloc[num]['quaternion']
    tilt_corrected_quaternion = head_orientation_tilt_correction.iloc[num]['quaternion']
    yaw_corrected_quaternion = head_orientation_yaw_correction.iloc[num]['quaternion']
    
    axes = [ax1, ax2, ax3]
    quats = [quaternion, tilt_corrected_quaternion, yaw_corrected_quaternion]
    
    for i in range(3):
        ax = axes[i]
        quat = quats[i]

        x_vec = quaternion_rotation(quat, x_proj_vec)
        y_vec = quaternion_rotation(quat, y_proj_vec)
        z_vec = quaternion_rotation(quat, z_proj_vec)

        axisX[i] = ax.quiver(0, 0, 0, *x_vec[1:], color='Blue', label='X')
        axisY[i] = ax.quiver(0, 0, 0, *y_vec[1:], color='Orange', label='Y')
        axisZ[i] = ax.quiver(0, 0, 0, *z_vec[1:], color='Green', label='Z')
        
        ax.legend(loc='center left')

fig = plt.figure(figsize=(24, 8))
fig.canvas.set_window_title('Orientation Tracking - Half speed')
# Gyroscope integration
ax1 = fig.add_subplot(131, projection='3d')
ax1.set_title("Gyroscope integration")
ax1.set_xlim3d([-1.0*scale, 1.0*scale]); ax1.set_xlabel('X')
ax1.set_ylim3d([-1.0*scale, 1.0*scale]); ax1.set_ylabel('Y')
ax1.set_zlim3d([-1.0*scale, 1.0*scale]); ax1.set_zlabel('Z')

# Gyroscope + Accelerometer
ax2 = fig.add_subplot(132, projection='3d')
ax2.set_title("Gyroscope + Accelerometer")
ax2.set_xlim3d([-1.0*scale, 1.0*scale]); ax2.set_xlabel('X')
ax2.set_ylim3d([-1.0*scale, 1.0*scale]); ax2.set_ylabel('Y')
ax2.set_zlim3d([-1.0*scale, 1.0*scale]); ax2.set_zlabel('Z')

# Gyroscope + Accelerometer + Magnetometer
ax3 = fig.add_subplot(133, projection='3d')
ax3.set_title("Gyroscope + Accelerometer + Magnetometer")
ax3.set_xlim3d([-1.0*scale, 1.0*scale]); ax3.set_xlabel('X')
ax3.set_ylim3d([-1.0*scale, 1.0*scale]); ax3.set_ylabel('Y')
ax3.set_zlim3d([-1.0*scale, 1.0*scale]); ax3.set_zlabel('Z')

ani = animation.FuncAnimation(fig, animate_axes, N, fargs=(ax1, ax2, ax3), interval=interval, blit=False)

plt.tight_layout()
plt.show()

In [ ]:
# Problem 6

# Gyroscope Integration
head_position = pd.DataFrame(columns=['time', 'posGlobal', 'quatBody', 'velocity', 'quaternion', 'W', 'X', 'Y', 'Z'])
# Copy data from the simple head orientation estimate
for col in ['time', 'quaternion', 'W', 'X', 'Y', 'Z']:
    head_position[col] = head_orientation[col][:]

# Gyroscope + Accelerometer
head_position_tilt = pd.DataFrame(columns=['time', 'posGlobal', 'quatBody', 'velocity', 'quaternion', 'W', 'X', 'Y', 'Z'])
# Copy data from the tilt corrected head orientation estimate
for col in ['time', 'quaternion', 'W', 'X', 'Y', 'Z']:
    head_position_tilt[col] = head_orientation_tilt_correction[col][:]

# Gyroscope + Accelerometer + Magnetometer
head_position_yaw = pd.DataFrame(columns=['time', 'posGlobal', 'quatBody', 'velocity', 'quaternion', 'W', 'X', 'Y', 'Z'])
# Copy data from the yaw corrected head orientation estimate
for col in ['time', 'quaternion', 'W', 'X', 'Y', 'Z']:
    head_position_yaw[col] = head_orientation_yaw_correction[col][:]

l_n = 0.15 # Distance from height of pupils to the base of the neck (0.15m)
l_t = 0.8 # Distance from the base of the neck to the waist (0.8m)

def positional_tracking(prev_pos_row, current_pos_row, current_IMU_Data_row):
    # Extract the raw accelerometer readings from the IMU_Data row
    # Raw accelerometer readings are used to allow a_l to more easily be extracted from a by subtracting 1 from a_l (as accelerometer measures in terms of g)
    accX = current_IMU_Data_row['raw_accX']
    accY = current_IMU_Data_row['raw_accY']
    accZ = current_IMU_Data_row['raw_accZ']
    
    # Extract data from prev_pos_row
    q = current_pos_row['quaternion'] # head orientation
    prev_vel = prev_pos_row['velocity']
    q1_prev = prev_pos_row['quatBody'] # quaternion describing the body orientation
    
    dt = current_IMU_Data_row['time'] - prev_pos_row['time'] # Time since last timestep
    
    # Convert accelerometer readings to global frame
    acc = (0, accX, accY, accZ)
    
    a_g = quaternion_rotation(q, acc) # Acceleration in global frame

    # Subtract 1 (gravity) from Z acceleration to get (estimated) linear acceleration
    a_g = (a_g[1], a_g[2], a_g[3] - 1)

    # Double integrate global accelerometer readings to produce body position
    
    # 1. Do this by converting linear acceleration into change in angular velocity as omega = (acc_linear * t)/l_t, where l_t is the length of the body
    #    (this makes the assumption that the linear acceleration is caused by a rotation of the body, not of the neck)
    # 2. Then, add this to the previous angular velocity to get the current angular velocity
    # 3. Then, apply simple dead reckoning to get an estimate for orientation of the body
    # 4. Lastly, compose this estimate with the estimate for the head position produced by the simple kinematic head model.
    # This produces a estimated global position based on double integration of acceleration and a kinematic model of the body (and head).
    
    # 1. Linear acc to angular velocity conversion
    omega = list(map(lambda acc: acc*dt/l_t, a_g)) # omega = angular velocity of body
    
    # 2. Add the previous velocity
    omega = [omega[0] + prev_vel[0], omega[1] + prev_vel[1], omega[2] + prev_vel[2]]
    
    # 3. Apply simple dead reckoning filter
    
    # Normalize the angular velocity
    mag_omega = np.sqrt(omega[0]**2 + omega[1]**2 + omega[2]**2)
    omega_normed = list(map(lambda ang_vel: ang_vel/mag_omega, omega))
    
    # Calculate the angle of the rotation
    theta = mag_omega*dt
    
    # Create the change quaternion from this axis-angle representation
    d_q = (np.cos(theta/2), omega_normed[0]*np.sin(theta/2), omega_normed[1]*np.sin(theta/2), omega_normed[2]*np.sin(theta/2))

    # Produce the current torso orientation by combining (multiplying) the last orientation and the change quaternion
    q1 = quaternion_product(q1_prev, d_q)
    
    # Rotate (0, 0, l_t) by the torso orientation quaternion
    r1 = quaternion_rotation(q1, (0, 0, 0, l_t))[1:] # torso position
    
    # Get the estimate for head position from simple kinematic head model
    # Apply kinematic head model to produce head position
    r = (0, 0, 0, l_n) # head position
    r = quaternion_rotation(q, r)[1:] # head position after simple kinematic head model
    
    # Compose body and head position to produce the final position
    p = np.add(r1, r) # global position
    
    # Return all required values
    return p, q1, omega

# Initial values - position = (0, 0, ln), body position = (0, 0, 0), velocity = (0, 0, 0)
for table in [head_position, head_position_tilt, head_position_yaw]:
    table.at[0, 'posGlobal'] = (0, 0, l_n+l_t)
    table.at[0, 'quatBody'] = (1, 0, 0, 0)
    table.at[0, 'velocity'] = (0, 0, 0)

for index in range(len(head_position)):
    # Skip the first one as this has the value of the identity quaternion
    if index != 0:
        prev_row = head_position.iloc[index - 1]
        cur_row = head_position.iloc[index]
        imu_row = IMU_Data.iloc[index]
        # Estimate the current head position, with tilt correction
        pos_global, posBody, velocity = positional_tracking(prev_row, cur_row, imu_row)
        head_position.at[index, 'posGlobal'] = pos_global
        head_position.at[index, 'quatBody'] = posBody
        head_position.at[index, 'velocity'] = velocity

# Gyroscope + Accelerometer
for index in range(len(head_position_tilt)):
    # Skip the first one as this has the value of the identity quaternion
    if index != 0:
        prev_row = head_position_tilt.iloc[index - 1]
        cur_row = head_position_tilt.iloc[index]
        imu_row = IMU_Data.iloc[index]
        # Estimate the current head position, with tilt correction
        pos_global, posBody, velocity = positional_tracking(prev_row, cur_row, imu_row)
        head_position_tilt.at[index, 'posGlobal'] = pos_global
        head_position_tilt.at[index, 'quatBody'] = posBody
        head_position_tilt.at[index, 'velocity'] = velocity

# Gyroscope + Accelerometer + Magnetometer
for index in range(len(head_position_yaw)):
    # Skip the first one as this has the value of the identity quaternion
    if index != 0:
        prev_row = head_position_yaw.iloc[index - 1]
        cur_row =  head_position_yaw.iloc[index]
        imu_row = IMU_Data.iloc[index]
        # Estimate the current head position, with tilt correction
        pos_global, posBody, velocity = positional_tracking(prev_row, cur_row, imu_row)
        head_position_yaw.at[index, 'posGlobal'] = pos_global
        head_position_yaw.at[index, 'quatBody'] = posBody
        head_position_yaw.at[index, 'velocity'] = velocity

In [ ]:
# Problem 6 - 3D plots (full speed)

# Make plots pop-out and be interactive
%matplotlib qt

# 3D plots of the 3 main axis rotations + translations of the IMU for gyroscope integration, gyroscope + acceleration, gyroscope + acceleration + magnetometer

N = 6959 # Number of frames
interval = 4 # Time between frames in ms - 1000/256 as time steps are at 256Hz
resolution = 25 # How often frames are generated (i.e. resolution=4 means every 4th data point is plotted)
# The resolution variable is used to ensure the animation is full speed/half speed as required. With the given values it runs at the correct speeds on my computer, but if yours is faster/slower the actual speeds will likely differ

# Set the total number of frames as adjusted by resolution
N //= resolution
scale = 2

axisX, axisY, axisZ = [0, 0, 0], [0, 0, 0], [0, 0, 0]

def animate_axes(num, ax1, ax2, ax3):
    """Generate the new quivers"""
    global axisX, axisY, axisZ
    # Check that this isn't the first frame
    if axisX[0] != 0:
        for i in axisX, axisY, axisZ:
            for j in i:
                j.remove()
        
    num *= resolution
    
    x_proj_vec = (0, 1, 0, 0)
    y_proj_vec = (0, 0, 1, 0)
    z_proj_vec = (0, 0, 0, 1)
    # Apply the current quaternion (indexed by num) to these vectors
    quaternion = head_orientation.iloc[num]['quaternion']
    tilt_corrected_quaternion = head_orientation_tilt_correction.iloc[num]['quaternion']
    yaw_corrected_quaternion = head_orientation_yaw_correction.iloc[num]['quaternion']
    pos_simple = head_position.iloc[num]['posGlobal']
    pos_tilt   = head_position_tilt.iloc[num]['posGlobal']
    pos_yaw    = head_position_yaw.iloc[num]['posGlobal']
    
    axes = [ax1, ax2, ax3]
    quats = [quaternion, tilt_corrected_quaternion, yaw_corrected_quaternion]
    poss = [pos_simple, pos_tilt, pos_yaw]
    
    for i in range(3):
        ax = axes[i]
        quat = quats[i]
        pos = poss[i]
        
        x_vec = quaternion_rotation(quat, x_proj_vec)
        y_vec = quaternion_rotation(quat, y_proj_vec)
        z_vec = quaternion_rotation(quat, z_proj_vec)

        # Add the position tracking
        axisX[i] = ax.quiver(*pos, *x_vec[1:], color='Blue', label='X')
        axisY[i] = ax.quiver(*pos, *y_vec[1:], color='Orange', label='Y')
        axisZ[i] = ax.quiver(*pos, *z_vec[1:], color='Green', label='Z')
        
        ax.legend(loc='center left')

fig = plt.figure(figsize=(24, 8))
fig.canvas.set_window_title('Positional Tracking')
# Gyroscope integration
ax1 = fig.add_subplot(131, projection='3d')
ax1.set_title("Gyroscope integration")
ax1.set_xlim3d([-1.0*scale, 1.0*scale]); ax1.set_xlabel('X')
ax1.set_ylim3d([-1.0*scale, 1.0*scale]); ax1.set_ylabel('Y')
ax1.set_zlim3d([-1.0*scale, 1.0*scale]); ax1.set_zlabel('Z')

# Gyroscope + Accelerometer
ax2 = fig.add_subplot(132, projection='3d')
ax2.set_title("Gyroscope + Accelerometer")
ax2.set_xlim3d([-1.0*scale, 1.0*scale]); ax2.set_xlabel('X')
ax2.set_ylim3d([-1.0*scale, 1.0*scale]); ax2.set_ylabel('Y')
ax2.set_zlim3d([-1.0*scale, 1.0*scale]); ax2.set_zlabel('Z')

# Gyroscope + Accelerometer + Magnetometer
ax3 = fig.add_subplot(133, projection='3d')
ax3.set_title("Gyroscope + Accelerometer + Magnetometer")
ax3.set_xlim3d([-1.0*scale, 1.0*scale]); ax3.set_xlabel('X')
ax3.set_ylim3d([-1.0*scale, 1.0*scale]); ax3.set_ylabel('Y')
ax3.set_zlim3d([-1.0*scale, 1.0*scale]); ax3.set_zlabel('Z')

ani = animation.FuncAnimation(fig, animate_axes, N, fargs=(ax1, ax2, ax3), interval=interval, blit=False)

plt.tight_layout()
plt.show()

In [ ]:
# #REMOVEME

# x_yaw  = [i[0] for i in head_position_yaw['posGlobal']]
# y_yaw  = [i[1] for i in head_position_yaw['posGlobal']]
# z_yaw  = [i[2] for i in head_position_yaw['posGlobal']]
# x_tilt = [i[0] for i in head_position_tilt['posGlobal']]
# y_tilt = [i[1] for i in head_position_tilt['posGlobal']]
# z_tilt = [i[2] for i in head_position_tilt['posGlobal']]
# x_gyro = [i[0] for i in head_position['posGlobal']]
# y_gyro = [i[1] for i in head_position['posGlobal']]
# z_gyro = [i[2] for i in head_position['posGlobal']]


# figure(figsize=(12, 8))
# plt.plot(head_orientation_yaw_correction[time], x_yaw, label='X yaw')
# plt.plot(head_orientation_yaw_correction[time], y_yaw, label='Y yaw')
# plt.plot(head_orientation_yaw_correction[time], z_yaw, label='Z yaw')
# plt.plot(head_orientation_yaw_correction[time], x_tilt, label='X tilt')
# plt.plot(head_orientation_yaw_correction[time], y_tilt, label='Y tilt')
# plt.plot(head_orientation_yaw_correction[time], z_tilt, label='Z tilt')
# plt.plot(head_orientation_yaw_correction[time], x_gyro, label='X gyro')
# plt.plot(head_orientation_yaw_correction[time], y_gyro, label='Y gyro')
# plt.plot(head_orientation_yaw_correction[time], z_gyro, label='Z gyro')

# plt.axhline(0, color='black', alpha=0.3)
# plt.title("Position Tracking Global Positions")
# plt.xlabel("Time (s)")
# plt.ylabel("Position (m)")
# plt.legend(loc='upper right')
# plt.show()